In [3]:

import csv
import pandas as pd
import re



In [4]:
##this function accept a string with format:
## word1 ORGANIATION word2 LOCATION word3 PERSON
## and a input of required word type: LOCATION, PERSON, or ORGANIZATION
## and filter out a list with all required words(without type name)
def stripKeyWords(NERout, ReqWordType):
    splitWords=['LOCATION','PERSON','ORGANIZATION']
    x = NERout.split()

    listFiltered=[]
    for i in range(len(x)):
        filtered=""
        ##if found the required word type
        if(x[i]==ReqWordType):
            j=i-1
            ##look for the previous key words(one of the splitting words)
            ##put the index in j
            while x[j] not in splitWords:
                j=j-1
            ##add the words to filtered
            while j<i-1:
                j=j+1
                if filtered == '':
                    filtered=x[j]
                else:
                    filtered=filtered+' '+x[j]
            ##add to a list of filtered words
            listFiltered=listFiltered+[filtered]
    return listFiltered

In [5]:
## this block extract all data to a double list
with open('NERoutput.csv', newline='') as f:
    df = pd.read_csv(f)

with open('FireIncidentCommander.csv', newline='') as f1:
    df1 = pd.read_csv(f1)

allDataF=[]
allNames=[]
##[[fire name, commander, commander, commander...][firename 2, commander, commander...]]
for index, row in df1.iterrows():
    dataFramObj=[]
    dataFramObj= dataFramObj+[row['fire Name']]
    if(isinstance(row['Incident Commander'], float) == False):
        inciComString=row['Incident Commander']
        inciComToList= inciComString.split(';')
        print(inciComToList)
        for i in inciComToList:
            smallList=i.split(',')
            dataFramObj= dataFramObj+[smallList[0]]
    if(isinstance(df.iloc[index]['NER Name Extracted'], float) == False):
        separateNames=df.iloc[index]['NER Name Extracted']
        separateNames=re.sub("[(]","",separateNames)
        separateNames=re.sub("[)]","",separateNames)
        separateNames=re.sub("[,]"," ",separateNames)
        separateNames=re.sub("[']","",separateNames)
        listFiltered=dataFramObj+stripKeyWords(separateNames,'PERSON')
        for i in listFiltered:
            if(i!=row['fire Name'] and i not in dataFramObj):
                dataFramObj= dataFramObj+[i]
    allDataF=allDataF + [dataFramObj]
print(allDataF[6])

['Arapaho and Roosevelt National Forests']
['Canyon Lakes Ranger District']
['USDA Forest Service & Colorado Parks and Wildlife']
['Tyler Knight RXB3']
['Sequoia & Kings Canyon National Parks - Cremers / Loyd(t)']
['Brent Davidson - Type 3 Incident Commander/Rigo Flores- Incident Commander Trainee']
['Matt Sigg']
['Durand Jaramillo, USFS']
['B. Krueger']
['Shasta-Trinity National Forest']
['BLM Eagle Lake Field office']
['Jeremy McMahon, BLM Eagle Lake Field Office']
['M. Moe']
['Ball', ' J. Tennant (T)']
['MT DNRC']
['Arguinchona']
['Type 4 Incident Commander']
['Willy, Meyer (T)']
['Willy, Meyer (T)']
['TJ Green & Seth LaRowe']
['Humboldt-Toiyabe National Forest']
['Wise River Fire Department']
['Robert Smith']
['Rod Weeks IC/Jerrod Scheffelmaier IC(T)']
['Gillespie']
['Shane Martin']
['IC-Dane Johnson, IC Trainee-Jonathan Barrett']
['Zach Palmer, BLM']
['Lyndsay Fonger']
['Micah Fey']
['USFS Richard Baker']
['Matthew Kasovich']
['Texas A&M Forest Service']
['Texas A&M Forest Service

In [17]:
##extracting the column names
listColumnNames=['fire / person']
for i in allDataF:
    for j in i:
        if j!=i[0] and j not in listColumnNames:
            listColumnNames=listColumnNames+[j]

In [18]:
##initializing a list of all rows
allRows=[]
##initialize allRows with numFire*numAttribute size
for i in allDataF:
    row=[i[0]]
    for j in listColumnNames:
        row=row+[0]
    allRows=allRows+[row]
print(allDataF[0])


['Slash Pile Burning 2021', 'Arapaho and Roosevelt National Forests', 'Lump Gulch', 'Keyser RidgeSimpson', 'Peak Lump Gulch', 'Vasquez', 'Upper Fraser', 'Fraser', 'PeakLump Gulch', 'Lyons', 'Gilpin']


In [19]:
#add the ones
print(len(listColumnNames))
for i in range(len(allDataF)):
    j=0
    while j<len(allDataF[i])-1:
        if(allDataF[i][j]!=allRows[i][0] and allDataF[i][j]!=''):
            allRows[i][listColumnNames.index(allDataF[i][j])]=1
        j= j+1
    
    
    

982


In [20]:
file = open('personOutputTable.csv', 'w', newline ='')

with file:
    # identifying header  
    writer = csv.DictWriter(file, fieldnames = listColumnNames)
    writer.writeheader()

    # writing data row-wise into the csv file
    #goes for number of fire time
    for i in range(len(allRows)):
        inci={}
        for j in range(len(listColumnNames)):
            inci[listColumnNames[j]]=allRows[i][j]
        writer.writerow(inci)